In [1]:
using StatisticalRethinking, Distributed, JLD, LinearAlgebra
using Mamba

Data

In [2]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [3]:
df2 = filter(row -> row[:age] >= 18, df);

Input data for Mamba

In [4]:
data = Dict(
  :x => convert(Array{Float64,1}, df2[:weight]),
  :y => convert(Array{Float64,1}, df2[:height])
);

Log-transformed Posterior(b0, b1, log(s2)) + Constant and Gradient Vector

In [5]:
logfgrad = function(x::DenseVector)
  b0 = x[1]
  b1 = x[2]
  logs2 = x[3]
  r = data[:y] .- b0 .- b1 .* data[:x]
  logf = (-0.5 * length(data[:y]) - 0.001) * logs2 -
           (0.5 * dot(r, r) + 0.001) / exp(logs2) -
           0.5 * b0^2 / 1000 - 0.5 * b1^2 / 1000
  grad = [
    sum(r) / exp(logs2) - b0 / 1000,
    sum(data[:x] .* r) / exp(logs2) - b1 / 1000,
    -0.5 * length(data[:y]) - 0.001 + (0.5 * dot(r, r) + 0.001) / exp(logs2)
  ]
  logf, grad
end

# MCMC Simulation with No-U-Turn Sampling

n = 5000
burnin = 1000
sim = Mamba.Chains(n, 3, start = (burnin + 1), names = ["b0", "b1", "s2"])
theta = NUTSVariate([0.0, 0.0, 0.0], logfgrad)
for i in 1:n
  sample!(theta, adapt = (i <= burnin))
  if i > burnin
    sim[i, :, 1] = [theta[1:2]; exp(theta[3])]
  end
end

Summarize draws

In [6]:
describe(sim)#-

Iterations = 1001:5000
Thinning interval = 1
Chains = 1
Samples per chain = 4000

Empirical Posterior Estimates:
       Mean          SD       Naive SE      MCSE         ESS   
b0 113.59676987 1.981435912 0.031329253 0.1129705832  307.63045
b1   0.91130375 0.043551391 0.000688608 0.0024605777  313.27800
s2  26.14311988 2.118477836 0.033496076 0.0528439599 1607.15269

Quantiles:
       2.5%       25.0%       50.0%        75.0%        97.5%   
b0 110.0166499 112.2275654 113.4510118 114.85846141 117.75469506
b1   0.8178253   0.8833593   0.9146177   0.94096635   0.99018456
s2  22.3062104  24.7459594  25.9899771  27.43171301  30.56048101



*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*